In [1]:
import numpy as np
import pandas as pd
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
 
nltk.download('stopwords')
nltk.download('word_tokenize')
nltk.download('punkt')

#nltk.data.path.append("C:\\Users\\DRM\AppData\\Roaming\\nltk_data")

dataset = pd.read_csv("C:\\temporario\\api6_fatec\\sprint2\\ml\\csv_completo.csv")

#print(dataset.head(10))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DRM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DRM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\DRM\AppData\Local\Temp\ipykernel_14260\1665798328.py:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("C:\\temporario\\api6_fatec\\sprint2\\ml\\csv_completo.csv")


In [3]:
#print(dataset.columns)
colunas_para_excluir = ["submission_date", "reviewer_id", "product_id", "product_name","product_brand","site_category_lv2","review_title","overall_rating","recommend_to_a_friend","reviewer_birth_year", "reviewer_gender","Feeling_True"]
dataset = dataset.drop(colunas_para_excluir, axis=1)

# Remover linhas com valores nulos
dataset = dataset.dropna()

print(dataset.head(1))

  site_category_lv1                                        review_text  \
0       Informática  Estou contente com a compra entrega rápida o ú...   

  reviewer_state Feeling_Predicted  
0             RJ          Positive  


In [4]:
 

# Função para processar o texto
def process_text(texts):
    all_tokens = []

     # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    additional_stopwords = {'produto', 'para', 'pra', 'comprei', 'ainda', 'tudo', 'bem', 'chegou', 'pois', 'não','nao', 'pode', 'mais', 'mas', 'dia', 'duas', 'um', 'ter', 'outro', 'dias', 'ar', 'agora', 'demais', 'muito', 'muita', 'tudo', 'todos', 'vem', 'ficar', 'veio', 'fazer', 'ficar', 'ficou', 'saber', 'porque', 'mim', 'me', 'lhe', 'vez', 'eau', 'achei', 'fiquei', 'senti', 'comprar', 'comprei', 'comprou', 'compra', 'vcs','vc', 'vezes', 'vez', 'cm', 'ja','já', 'posso', '15','1','2','3','4','5','6','7','8','9','10','jeito', 'mmmmmmmmmmmmmmmmmmmmmmmmmmmmmm','dar','so','só','porém'}
    stop_words.update(additional_stopwords)

    common_categories = ['ele', 'ela', 'eles', 'elas', 'um', 'uma', 'uns', 'umas', 'o', 'a', 'os', 'as', 'de', 'em', 'para', 'com', 'sem', 'sobre', 'entre', 'antes', 'depois', 'através', 'de', 'do', 'das', 'dekas', 'deles', 'da','entre', 'todos', 'todas', 'quando', 'como','junto']
    stop_words.update(common_categories)

    
    for text in texts:
        # Tokenizar o texto em palavras
        tokens = word_tokenize(text.lower())       
        filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
        all_tokens.extend(filtered_tokens)
    
    return all_tokens

# Dicionário para armazenar as informações de cada grupo e as palavras mais comuns
top_words_by_group = {}

for name, group in dataset.groupby(['site_category_lv1', 'reviewer_state', 'Feeling_Predicted']):
    texts = group['review_text']
    processed_tokens = process_text(texts)
    word_counts = Counter(processed_tokens)
    # Selecionar as 5 palavras mais comuns
    top_words = word_counts.most_common(10)
    top_words_by_group[name] = {'top_words': top_words}
    

In [9]:

 

# Criar uma lista para armazenar os dados
data = []

# Iterar sobre o dicionário e extrair as informações necessárias
for group_info, contents in top_words_by_group.items():
    # group_info contém a tupla com as informações do grupo
    # contents contém outro dicionário com as palavras mais comuns
    top_words = contents['top_words']
    words = []
    for word, count in top_words:
        words.append({'word':word, 'count':count})

    data.append({
            'site_category_lv1': group_info[0],
            'reviewer_state': group_info[1],
            'Feeling_Predicted': group_info[2],
            'top_words': words 
        })

# Converter a lista de dicionários em um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame como CSV
df.to_csv('top_words_by_group.csv', index=False,sep=';')
#print(df)